90 【支付宝一面：请手撕Self Attention - AI有温度 | 小红书 - 你的生活指南】 😆 oLhkJKU7IHnKjsL 😆 https://www.xiaohongshu.com/discovery/item/67ae8ff500000000180088b9?source=webshare&xhsshare=pc_web&xsec_token=ABePf-YJEce9jJC7EAe_sXnW29guXqgcFjbV0Y1fwe-ws=&xsec_source=pc_share

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self,embed_size,h,dropout):
        super().__init__()
        self.embed_size = embed_size
        self.h = h  # 多头注意力的数量

        assert embed_size % h == 0, "Embedding size should be divisible by the number of heads"

        self.embed_size_per_head = embed_size // h  # 每个头的embedding大小
        self.query = nn.Linear(embed_size,embed_size)
        self.key = nn.Linear(embed_size,embed_size)
        self.value = nn.Linear(embed_size,embed_size)
        self.attention_dropout = nn.Dropout(dropout)  # 添加dropout以防止过拟合
        self.output_linear = nn.Linear(embed_size,embed_size)
    
    def forward(self,datas,mask=None):
        Q = self.query(datas) #(batch_size, seq_len, embed_size) -> (batch_size, seq_len, embed_size)
        K = self.key(datas)   #(batch_size, seq_len, embed_size) -> (batch_size, seq_len, embed_size)  
        V = self.value(datas) #(batch_size, seq_len, embed_size) -> (batch_size, seq_len, embed_size)
        
        Q = Q.view(Q.shape[0],Q.shape[1],self.h,self.embed_size_per_head).transpose(1,2)# (batch, seq_len, h, embed_size_per_head) -> (batch, h, seq_len, embed_size_per_head)
        K = K.view(K.shape[0],K.shape[1],self.h,self.embed_size_per_head).transpose(1,2)# (batch, seq_len, h, embed_size_per_head) -> (batch, h, seq_len, embed_size_per_head)
        V = V.view(V.shape[0],V.shape[1],self.h,self.embed_size_per_head).transpose(1,2)# (batch, seq_len, h, embed_size_per_head) -> (batch, h, seq_len, embed_size_per_head)

        # Q:(batch, h, seq_len, embed_size_per_head)
        # K.transpose:(batch, h, embed_size_per_head, seq_len)
        P = torch.matmul(Q,K.transpose(-2,-1)) 
        # P: (batch_size, h,seq_len, seq_len)
        attention_scores = P/torch.sqrt(torch.tensor(self.embed_size))
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
        
        # 归一化
        attention_weight = F.softmax(attention_scores, dim=-1)
        attention_weight = self.attention_dropout(attention_weight)  # 应用dropout
        # attention_weight: (batch_size, h,seq_len, seq_len) V:(batch, h, seq_len, embed_size_per_head) -> （batch_size, seq_len, embed_size_per_head)
        x = torch.matmul(attention_weight,V)
        # x: (batch_size,h,seq_len,self.embed_size_per_head) -> (batch_size, h,seq_len, embed_size_per_head) -> (batch_size, seq_len, embed_size)
        x = x.transpose(1,2).contiguous().view(x.shape[0],-1,self.embed_size)
        # x: (batch_size, seq_len, embed_size)
        output = self.output_linear(x)  # 线性变换以得到最终的输出
        return output


In [5]:
datas = torch.rand(3,2,4)
mask = torch.ones(3,2,2)
net = SelfAttention(4,4,0.1)
output = net(datas,None)
print(output.shape)

torch.Size([3, 2, 4])


In [6]:
torch.tril(torch.ones(2,2),diagonal=0)

tensor([[1., 0.],
        [1., 1.]])